In [6]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from typing import TypedDict
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)
model = ChatHuggingFace(llm=llm)

In [8]:
# Create a state

class LLMState(TypedDict):
    question: str
    answer: str
    


In [9]:
def llm_qa(state: LLMState) -> LLMState:
    
    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f'Answer the folowing question {question}'

    # Ask that question to the LLM
    answer = model.invoke(prompt).content

    # Update the answer
    state['answer'] = answer

    return state

In [13]:
# create our graph
graph = StateGraph(LLMState)

# add_nodes
graph.add_node('llm_qa', llm_qa)

# Add edges

graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile

workflow = graph.compile()

In [14]:
# Execute

initial_state = {'question': 'How far is moon from earth?'}

final_state = workflow.invoke(initial_state)

In [15]:
print((final_state))

{'question': 'How far is moon from earth?', 'answer': "The distance between the Earth and the moon is not constant because the moon orbits the Earth in an elliptical path. \n\nHere's a breakdown:\n\n* **Average distance:**  Around **238,855 miles (384,400 kilometers)**\n* **Perigee (closest point):** ~225,623 miles (363,104 kilometers)\n* **Apogee (farthest point):** ~252,088 miles (405,696 kilometers)\n\nSo, the distance between the Earth and the moon changes, but on average, it's about 238,855 miles! \n"}
